In [57]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from pydantic import BaseModel,Field
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from langchain.agents import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
load_dotenv()


True

In [8]:
os.environ["OPENAI_API_KEY"] = os.getenv("Open_api_key2")

In [9]:
model=ChatOpenAI(temperature=0.5)

In [49]:
class personinfo(BaseModel):
    """"Get the information of a person"""
    name:str=Field(...,title="Name of the person")
    age:int=Field(...,title="Age of the person")
class getweather(BaseModel):
    """Get the weather of a city"""
    
    city:str=Field(...,title="City name")
        


In [82]:
@tool(args_schema=personinfo)
def get_person_info(name:str,age:int):
    return f"The name of the person is {name} and the age is {age}"
@tool(args_schema=getweather)
def get_weather(city:str):
    return f"The weather of {city} is 25 degree celcius"

In [83]:
openai_function=[format_tool_to_openai_function(f) for f in [
    get_person_info,
    get_weather,
]]

In [84]:
binded_model=model.bind(functions=openai_function)

In [85]:
prompt=ChatPromptTemplate.from_messages(
    {
        ("system","you are helpful assistant your duty is to get the infomation of person asking"),
        ("user","{input}")
    }
)

In [86]:
responce=prompt | binded_model | OpenAIFunctionsAgentOutputParser()

In [87]:
result=responce.invoke({"input":"people like to visit karachi"})

In [88]:
get_weather.run(result.tool_input)

'The weather of karachi is 25 degree celcius'

In [89]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result,AgentFinish):
        print(result)
    else:
        tools={
            "get_person_info":get_person_info,
            "get_weather":get_weather
        }
        return tools[result.tool].run(result.tool_input)

In [90]:
responce=prompt | binded_model | OpenAIFunctionsAgentOutputParser() | route

In [92]:
responce.invoke({"input":"people over there are basit and he is 25"})

'The name of the person is basit and the age is 25'